# Creazione modello text analysis
Ho visto che il decision tree è il modello che mi offre migliori performance, soprattutto con TfIdf e tokenizzazione nltk/regex con stemming PorterStemmer().


In [1]:
from my_tokenizer import MyTokenizer
#from classification import Classification
#from model import Model
import pandas as pd
import util_strings as utils

È necessaria la normalizzazione delle feature, dato che nel testo abbiamo i nomi delle squadre ed i soprannomi, vado a sostituire tutte le parole relative alla squadra di casa con home team e stessa cosa per la away team.
Questo viene fatto tramite il metodo feature_normalization()

In [2]:
mt = MyTokenizer(pd.read_csv(utils.completed_dataset, index_col=0))
#mt.feature_normalization()
mt.dataset

,home,date,result,goals,away,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,season,rank_h,rank_a,description,prediction
0,fiorentina,2019-08-24,P,-2.4,napoli,-7.2,-2.2,-0.2,0.0,-6.8,4.6,0.0,0.2,2019-2020,1.0,2.0,All eyes will be on Stadio Artemio Franchi whe...,P
1,parma,2019-08-24,P,0.8,juventus,-2.0,-0.2,0.2,0.2,-14.6,1.6,0.0,0.2,2019-2020,0.0,5.0,Juventus begin the defence of their Serie A tr...,P
2,torino,2019-08-25,V,0.8,sassuolo,1.2,2.4,0.2,0.4,2.2,2.4,-1.0,-0.6,2019-2020,3.0,0.0,Sassuolo will be looking to build on a hard-fo...,N
3,cagliari,2019-08-25,P,-1.0,brescia,5.0,3.0,-1.0,-1.0,18.0,-2.0,0.0,0.0,2019-2020,6.0,0.0,Brescia return to Serie A following an eight-y...,V
4,roma,2019-08-25,N,0.8,genoa,4.6,2.6,-0.2,-0.4,5.2,-3.6,0.2,-0.2,2019-2020,4.0,0.0,Roma kick off the new Serie A campaign with a ...,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,napoli,2022-10-16,V,1.4,bologna,5.2,3.2,0.4,0.4,6.2,-2.6,-0.6,0.0,2022-2023,9.0,2.0,"After rolling over Cremonese 4-1 on the road, ...",V
1223,lazio,2022-10-16,N,0.2,udinese,-4.4,0.0,0.2,0.4,-5.4,-5.0,-1.2,0.0,2022-2023,8.0,1.0,After rolling over Fiorentina 4-0 at the weeke...,V
1224,spezia,2022-10-16,N,0.2,cremonese,-1.4,-0.2,-0.2,-0.2,3.6,-0.6,0.8,0.0,2022-2023,4.0,0.0,Sunday’s football game at Stadio Alberto Picco...,V
1225,sampdoria,2022-10-17,P,-0.4,roma,-5.6,-0.6,-0.2,-0.4,-2.8,3.6,1.8,0.0,2022-2023,3.0,6.0,After sharing the spoils with Bologna on Dejan...,N


In [3]:
mt.clean_text()
X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()
mt.save_vectorizer(utils.vectorizer)
mt.dataset

,home,date,result,goals,away,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,season,rank_h,rank_a,description,prediction
0,fiorentina,2019-08-24,P,-2.4,napoli,-7.2,-2.2,-0.2,0.0,-6.8,4.6,0.0,0.2,2019-2020,1.0,2.0,"[(all, eye), (eye, stadio), (stadio, artemio),...",P
1,parma,2019-08-24,P,0.8,juventus,-2.0,-0.2,0.2,0.2,-14.6,1.6,0.0,0.2,2019-2020,0.0,5.0,"[(juventu, begin), (begin, defenc), (defenc, s...",P
2,torino,2019-08-25,V,0.8,sassuolo,1.2,2.4,0.2,0.4,2.2,2.4,-1.0,-0.6,2019-2020,3.0,0.0,"[(sassuolo, look), (look, build), (build, hard...",N
3,cagliari,2019-08-25,P,-1.0,brescia,5.0,3.0,-1.0,-1.0,18.0,-2.0,0.0,0.0,2019-2020,6.0,0.0,"[(brescia, return), (return, seri), (seri, a),...",V
4,roma,2019-08-25,N,0.8,genoa,4.6,2.6,-0.2,-0.4,5.2,-3.6,0.2,-0.2,2019-2020,4.0,0.0,"[(roma, kick), (kick, new), (new, seri), (seri...",V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,napoli,2022-10-16,V,1.4,bologna,5.2,3.2,0.4,0.4,6.2,-2.6,-0.6,0.0,2022-2023,9.0,2.0,"[(after, roll), (roll, cremones), (cremones, 4...",V
1223,lazio,2022-10-16,N,0.2,udinese,-4.4,0.0,0.2,0.4,-5.4,-5.0,-1.2,0.0,2022-2023,8.0,1.0,"[(after, roll), (roll, fiorentina), (fiorentin...",V
1224,spezia,2022-10-16,N,0.2,cremonese,-1.4,-0.2,-0.2,-0.2,3.6,-0.6,0.8,0.0,2022-2023,4.0,0.0,"[(sunday, ’), (’, footbal), (footbal, game), (...",V
1225,sampdoria,2022-10-17,P,-0.4,roma,-5.6,-0.6,-0.2,-0.4,-2.8,3.6,1.8,0.0,2022-2023,3.0,6.0,"[(after, share), (share, spoil), (spoil, bolog...",N


## Classificazione del modello
Migliori risultati con Decision Tree.
Viene salvato il modello

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model = DecisionTreeClassifier()

In [7]:
import sklearn.metrics as mtr
E = []

accuracy = 0
while(accuracy < 0.782):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    analysis = pd.DataFrame()
            
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    if report['accuracy'] > accuracy: 
        accuracy = report['accuracy']
        print(accuracy)

0.7560975609756098
0.7886178861788617


In [8]:
analysis = {
    'Model': 'Model', 'Accuracy': report['accuracy'],
    'Avg Precision (macro)': report['macro avg']['precision'],
    'Avg Recall (macro)': report['macro avg']['recall'],
    'Avg F1-score (macro)': report['macro avg']['f1-score'],
    'Avg Precision (weighted)': report['weighted avg']['precision'],
    'Avg Recall (weighted)': report['weighted avg']['recall'],
    'Avg F1-score (weighted)': report['weighted avg']['f1-score']
}
analysis

{'Model': 'Model',
 'Accuracy': 0.7886178861788617,
 'Avg Precision (macro)': 0.7825703134160845,
 'Avg Recall (macro)': 0.7467776306251807,
 'Avg F1-score (macro)': 0.7582134964555762,
 'Avg Precision (weighted)': 0.787139567008515,
 'Avg Recall (weighted)': 0.7886178861788617,
 'Avg F1-score (weighted)': 0.7818101062021523}

In [9]:
import pickle

In [10]:
with open(utils.text_analysis, 'wb') as file:
    pickle.dump(model, file)